In [ ]:
import os
import pandas as pd

def read_all_csvs(main_path, dataset_names, classifier_names, algorithm_names, date):
    dfs_fairness = []
    dfs_performance = []
    
    for dataset_name in dataset_names:
        df_data_fair = []
        df_data_perf = []
        for classifier_name in classifier_names:
            for algorithm_name in algorithm_names:
                folder_path = os.path.join(main_path, f'{algorithm_name}_{dataset_name}_{classifier_name}')
                folders_inside_folder = [f for f in os.listdir(folder_path) if 'hvdm' in f] #'heom' in f and '_1' not in f and '_2' not in f and '_3' not in f and '_4' not in f]
                #print(folders_inside_folder)
                folders_inside_folder = [f for f in os.listdir(os.path.join(folder_path, 'hvdm'))]
                for folder_name in folders_inside_folder:
                    if algorithm_name in ['fawos', 'hfos', 'fos']:
                        # f = [f for f in os.listdir(os.path.join(folder_path, folder_name)) if "5" in f ][0]
                        # print(f)
                        folder_name = os.path.join('hvdm', folder_name)
                        #folder_name = os.path.join(folder_name, f)
                    for i in range(0, 5):
                        if os.path.exists(os.path.join(folder_path, folder_name, date, f'fairness_{i}.csv')):
                            fair_path = os.path.join(folder_path, folder_name, date, f'fairness_{i}.csv')
                            perf_path = os.path.join(folder_path, folder_name, date, f'performance_{i}.csv')
                            df_fair = pd.read_csv(fair_path)
                            df_performance = pd.read_csv(perf_path)
                            df_fair['classifier'] = [classifier_name] * len(df_fair)
                            df_fair['algorithm'] = [algorithm_name if 'fair' in x else '-' for x in df_fair['data']]
                            df_fair['iteration'] = [i] * len(df_fair)
                            df_fair['params'] = [folder_name if 'fair' in x else '-' for x in df_fair['data']]
                            # df_fair.drop(columns=['data', 'average_odds', 'average_absolute_odds'], inplace=True)
                            df_fair.drop(columns=['data'], inplace=True)
                            df_performance['classifier'] = [classifier_name] * len(df_performance)
                            df_performance['algorithm'] = [algorithm_name if 'fair' in x else '-' for x in df_performance['data']]
                            df_performance['iteration'] = [i] * len(df_performance)
                            df_performance['params'] = [folder_name if 'fair' in x else '-' for x in df_performance['data']]
                            df_performance.drop(columns=['data'], inplace=True)
                            df_data_fair.append(df_fair)
                            df_data_perf.append(df_performance)
        dfs_fairness.append(df_data_fair)
        dfs_performance.append(df_data_perf)
    return dfs_fairness, dfs_performance

In [ ]:
def aggregate_runs(dfs):
    df = pd.concat(dfs).reset_index(drop=True)
    df.drop(columns=['iteration'], inplace=True)
    df_num_cols = [c for c in df.columns if c not in ['classifier', 'algorithm', 'params']]
    df.loc[:, df_num_cols] = df.loc[:, df_num_cols].abs()
    df = df.groupby(['classifier', 'algorithm', 'params']).agg(['mean', 'std'])
    return df

In [ ]:
def aggregate_everything(dfs):
    df = pd.concat(dfs).reset_index(drop=True)
    df.drop(columns=['iteration', 'classifier', 'algorithm'], inplace=True)
    df_num_cols = [c for c in df.columns if c not in ['params']]
    df.loc[:, df_num_cols] = df.loc[:, df_num_cols].abs()
    df = df.groupby(['params']).agg(['mean', 'std'])
    return df

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

def aggregate_runs_no_mean(dfs, y):
    df = pd.concat(dfs).reset_index(drop=True)
    df_num_cols = [c for c in df.columns if c not in ['classifier', 'algorithm', 'params']]
    #df.loc[:, df_num_cols] = df.loc[:, df_num_cols].abs()
    g = sns.FacetGrid(df, col="classifier")
    g.map_dataframe(sns.lineplot, x='iteration', y=y, hue='params', errorbar=None)
    g.add_legend()
    plt.show()

In [ ]:
import numpy as np
from scipy.stats import rankdata

main_paths = ['../validation', '../validation_multi']
datasets_names = [['german', 'heart_disease', 'adult', 'bank'], ['adult', 'german', 'bank']]
classifier_names = ['logistic_regression', 'mlp', 'decision_tree']
algorithm_names = ['hfos']
date = '2024-06-12'

df_sums_aao, df_sums_p = [], []
all_df_sums = []
for main_path, dataset_names in zip(main_paths, datasets_names):
    dfs_fairness, dfs_performance = read_all_csvs(main_path, dataset_names, classifier_names, algorithm_names, date)
    df_sums_aao_one = []
    df_sums_p_one = []
    all_df_sums0 = []
    for dataset_name, dfs_f, dfs_p in zip(dataset_names, dfs_fairness, dfs_performance):
        #display(aggregate_runs(dfs_f))
        df_f = aggregate_everything(dfs_f)
        df_p = aggregate_everything(dfs_p)
        
        df_sums1 = pd.concat(dfs_f).reset_index(drop=True)[['iteration', 'classifier', 'algorithm', 'params', 'average_absolute_odds']]
        df_sums1 = df_sums1[df_sums1['algorithm'] != '-']
        #print(df_sums1)
        df_sums1.set_index(['iteration', 'classifier', 'algorithm', 'params'], inplace=True)
        df_sums2 = pd.concat(dfs_p).reset_index(drop=True)[['iteration', 'classifier', 'algorithm', 'params', 'gmean']]
        df_sums2 = df_sums2[df_sums2['algorithm'] != '-']
        df_sums2.set_index(['iteration', 'classifier', 'algorithm', 'params'], inplace=True)
        df_sums = df_sums1.join(df_sums2)
        df_sums['score'] = ((1 - df_sums1['average_absolute_odds']) * df_sums2['gmean']) ** 0.5
        df_sums.reset_index(inplace=True)
        df_sums.drop(columns=['average_absolute_odds', 'gmean', 'iteration', 'classifier', 'algorithm'], inplace=True)
        df_sums.set_index('params', inplace=True)
        all_df_sums0.append(df_sums)
        all_df_sums.append(df_sums)
        
        print(dataset_name)
        display(df_f)
        display(df_p)
        df_sum = ((1 - df_f[('average_absolute_odds', 'mean')].abs()) * df_p[('gmean', 'mean')]) ** 0.5
        df_sums_aao.append(df_f[('average_absolute_odds', 'mean')].abs())
        df_sums_p.append(df_p[('gmean', 'mean')])
        df_sums_aao_one.append(df_f[('average_absolute_odds', 'mean')].abs())
        df_sums_p_one.append(df_p[('gmean', 'mean')])
        #print(df_sum)
        print('-- overall score')
        display(df_sums.groupby(['params']).mean())
        print('----------------------------------------------------------------')
    #df_sum = ((1 - pd.concat(df_sums_aao_one, axis=1).agg(['mean'], axis=1)) * pd.concat(df_sums_p_one, axis=1).agg(['mean'], axis=1)) ** 0.5
    print(pd.concat(df_sums_aao_one, axis=1).agg(['mean'], axis=1))
    print(pd.concat(df_sums_p_one, axis=1).agg(['mean'], axis=1))
    
    display(pd.concat(all_df_sums0, axis=0).groupby(['params']).mean())
    #print(df_sum)
    print('=============================================================================================')
print('Overall')
df_sum = ((1 - pd.concat(df_sums_aao, axis=1).agg(['mean'], axis=1)) * pd.concat(df_sums_p, axis=1).agg(['mean'], axis=1)) ** 0.5


#print(df_sums)
print(pd.concat(df_sums_aao, axis=1).agg(['mean'], axis=1))
print(pd.concat(df_sums_p, axis=1).agg(['mean'], axis=1))
print(df_sum)

display(pd.concat(all_df_sums, axis=0).groupby(['params']).mean())

In [ ]:
df_sums_aao = []
df_sums_p = []
for dataset_name, dfs_f, dfs_p in zip(dataset_names, dfs_fairness, dfs_performance):
    df_f = aggregate_everything(dfs_f)
    df_p = aggregate_everything(dfs_p)
    print(dataset_name)
    display(df_f)
    display(df_p)
    df_sum = ((1 - df_f[('average_absolute_odds', 'mean')].abs()) * df_p[('gmean', 'mean')]) ** 0.5
    df_sums_aao.append(df_f[('average_absolute_odds', 'mean')].abs())
    df_sums_p.append(df_p[('gmean', 'mean')])
    print(df_sum)
    print('----------------------------------------------------------------')
df_sum = ((1 - pd.concat(df_sums_aao, axis=1).agg(['mean'], axis=1)) * pd.concat(df_sums_p, axis=1).agg(['mean'], axis=1)) ** 0.5
print(pd.concat(df_sums_aao, axis=1).agg(['mean'], axis=1))
print(pd.concat(df_sums_p, axis=1).agg(['mean'], axis=1))
print(df_sum)

In [ ]:
for dataset_name, dfs in zip(dataset_names, dfs_performance):
    df = aggregate_everything(dfs)
    print(dataset_name)
    display(df)
    print('----------------------------------------------------------------')

In [ ]:
for dataset_name, dfs in zip(dataset_names, dfs_fairness):
    df = aggregate_runs(dfs)
    print(dataset_name)
    display(df)
    print('----------------------------------------------------------------')

In [ ]:
for dataset_name, dfs in zip(dataset_names, dfs_performance):
    df = aggregate_runs(dfs)
    print(dataset_name)
    display(df)
    print('----------------------------------------------------------------')

In [ ]:
for dataset_name, dfs in zip(dataset_names, dfs_fairness):
    print(dataset_name)
    aggregate_runs_no_mean(dfs, y='statistical_parity')